In [2]:
import numpy as np
from numpy.lib import stride_tricks
import os
from PIL import Image
import scipy.io.wavfile as wav
import librosa  
import errno
import glob
import csv

In [4]:
# samplerate, samples = wav.read("300_AUDIO_8000.wav")

In [5]:
# samplerate

In [6]:
# len(samples)

In [4]:
inputDir = os.getcwd() + "/participantsAudio"

if(not os.path.exists(inputDir)):
    print("Input Directoy Not Found")
else:
    listOfFile = os.listdir(inputDir)

listOfFile

['458_AUDIO_Participant.wav',
 '436_AUDIO_Participant.wav',
 '480_AUDIO_Participant.wav',
 '308_AUDIO_Participant.wav',
 '366_AUDIO_Participant.wav',
 '370_AUDIO_Participant.wav',
 '420_AUDIO_Participant.wav',
 '462_AUDIO_Participant.wav',
 '384_AUDIO_Participant.wav',
 '332_AUDIO_Participant.wav',
 '324_AUDIO_Participant.wav',
 '392_AUDIO_Participant.wav',
 '474_AUDIO_Participant.wav',
 '428_AUDIO_Participant.wav',
 '446_AUDIO_Participant.wav',
 '378_AUDIO_Participant.wav',
 '316_AUDIO_Participant.wav',
 '300_AUDIO_Participant.wav',
 '488_AUDIO_Participant.wav',
 '450_AUDIO_Participant.wav',
 '412_AUDIO_Participant.wav',
 '354_AUDIO_Participant.wav',
 '404_AUDIO_Participant.wav',
 '330_AUDIO_Participant.wav',
 '386_AUDIO_Participant.wav',
 '390_AUDIO_Participant.wav',
 '326_AUDIO_Participant.wav',
 '348_AUDIO_Participant.wav',
 '476_AUDIO_Participant.wav',
 '418_AUDIO_Participant.wav',
 '482_AUDIO_Participant.wav',
 '434_AUDIO_Participant.wav',
 '364_AUDIO_Participant.wav',
 '372_AUDI

In [5]:
outputDir = inputDir + "/output"
if (not os.path.exists(outputDir)):
    try:
        os.makedirs(outputDir)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise

listOfWavName = []
    
for item in listOfFile:
    if(item.endswith(".wav")):
        filename = item.split(".wav")[0]
        path = outputDir+"/"+filename
        listOfWavName.append(filename)
        if (not os.path.exists(path)):
            try:
                os.makedirs(path)
            except OSError as e:
                if e.errno != errno.EEXIST:
                    raise

In [9]:
def processFile(filename, file):
    samples, samplerate = librosa.load(filename, sr=8000)
    #INSERT LOOP FOR FRAMING 10s DATA
    totalSamples = len(samples)
    blockPerSegment = 80000
    cNumOfSegments = int(-(-totalSamples//blockPerSegment))
    fNumOfSegments = int(totalSamples//blockPerSegment)
    segments = []
    counter = 0

    for i in range(fNumOfSegments):
    #     segments.append(1)
        segments.append({(i)*blockPerSegment + (1 if i > 0 else 0), (i+1)*blockPerSegment})

    
    # segments
    for x, y in segments:
        if(x == 0):
            shortSamples = samples[x:y]
        else:
            shortSamples = samples[y:x]
        
        
        
        frameSize = 200
        hopsize = 80
        overlap = 120
        
        win = np.hanning(frameSize) #Forcuses on the peak (in the middle)
        s = np.append(np.zeros(overlap), shortSamples)
        # cols for windowing
        cols = np.ceil((len(s) - frameSize) / hopsize) + 1
        # zeros at end (thus samples can be fully covered by frames)
        s = np.append(s, np.zeros(overlap))
        
        frames = stride_tricks.as_strided(s, shape=(int(cols), frameSize),
                                          strides=(s.strides[0]*hopsize,
                                                   s.strides[0])).copy()
        frames *= win
        rfft = np.fft.rfft(frames)
        timebins, freqbins = np.shape(rfft)
        scale = np.linspace(0, 1, freqbins) ** 1
        scale *= (freqbins-1)/max(scale)
        scale = np.unique(np.round(scale))
        
        # create spectrogram with new freq bins
        newspec = np.complex128(np.zeros([timebins, len(scale)]))
        for i in range(0, len(scale)):
            if i == len(scale)-1:
                newspec[:, i] = np.sum(rfft[:, int(scale[i]):], axis=1)
            else:
                newspec[:, i] = np.sum(rfft[:, int(scale[i]):int(scale[i+1])], axis=1)
                
        # list center freq of bins
        allfreqs = np.abs(np.fft.fftfreq(freqbins*2, 1./8000)[:freqbins+1])
        
        freqs = []
        for i in range(0, len(scale)):
            if i == len(scale)-1:
                freqs += [np.mean(allfreqs[int(scale[i]):])]
            else:
                freqs += [np.mean(allfreqs[int(scale[i]):int(scale[i+1])])]
                
        ims = 20.*np.log10(np.abs(newspec)/10e-6)  # amplitude to decibel
        
        ims = np.transpose(ims)
        ims = np.flipud(ims)  # weird - not sure why it needs flipping
        
#         image = Image.new()
        image = Image.fromarray(ims)
        image = image.convert('L')  # convert to grayscale
        if(x == 0):
            image.save("participantsAudio/output/" + file + "/" + file + "("+ str(x) + "-"+ str(y) + ")"+".png")
        else:
            image.save("participantsAudio/output/" + file + "/" + file + "("+ str(y) + "-"+ str(x) + ")"+".png")
        image.close()

    
for item in listOfWavName:
    filepath = "participantsAudio/" + item + ".wav"
    processFile(filepath, item)

/Users/aaron/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in log10


In [10]:
# samples, samplerate = librosa.load('input/301_AUDIO.wav', sr=8000)

In [11]:
# samplerate

In [12]:
# len(samples)

In [13]:
# #INSERT LOOP FOR FRAMING 10s DATA
# totalSamples = len(samples)
# blockPerSegment = 80000
# cNumOfSegments = int(-(-totalSamples//blockPerSegment))
# fNumOfSegments = int(totalSamples//blockPerSegment)
# segments = []
# counter = 0

# for i in range(fNumOfSegments):
# #     segments.append(1)
#     segments.append({(i)*blockPerSegment + (1 if i > 0 else 0), (i+1)*blockPerSegment})

# # segments
# for x, y in segments:
#     newSamples = newSamples[x:y]

In [14]:
# samples = samples[:80000]
# #320000 = 40sec worth of audio samples

In [15]:
# frameSize = 200
# hopsize = 80
# overlap = 120

In [16]:
# len(samples)/samplerate

In [17]:
# win = np.hanning(frameSize) #Forcuses on the peak (in the middle)

In [18]:
# s = np.append(np.zeros(overlap), samples)
# # cols for windowing
# cols = np.ceil((len(s) - frameSize) / hopsize) + 1
# # zeros at end (thus samples can be fully covered by frames)
# s = np.append(s, np.zeros(overlap))

In [19]:
# cols

In [20]:
# frames = stride_tricks.as_strided(s, shape=(int(cols), frameSize),
#                                       strides=(s.strides[0]*hopsize,
#                                       s.strides[0])).copy()

In [21]:
# frames

In [22]:
# np.shape(frames)

In [23]:
# frames *= win
# #frames

In [24]:
# rfft = np.fft.rfft(frames)

In [25]:
# np.shape(rfft)

In [26]:
# timebins, freqbins = np.shape(rfft)

In [27]:
# timebins

In [28]:
# freqbins

In [29]:
# scale = np.linspace(0, 1, freqbins) ** 1
# scale *= (freqbins-1)/max(scale)
# scale = np.unique(np.round(scale))

In [30]:
# # create spectrogram with new freq bins
# newspec = np.complex128(np.zeros([timebins, len(scale)]))

In [31]:
# newspec

In [32]:
# for i in range(0, len(scale)):
#     if i == len(scale)-1:
#         newspec[:, i] = np.sum(rfft[:, int(scale[i]):], axis=1)
#     else:
#         newspec[:, i] = np.sum(rfft[:, int(scale[i]):int(scale[i+1])], axis=1)

In [33]:
# newspec

In [34]:
# # list center freq of bins
# allfreqs = np.abs(np.fft.fftfreq(freqbins*2, 1./8000)[:freqbins+1])

In [35]:
allfreqs

NameError: name 'allfreqs' is not defined

In [ ]:
# freqs = []
# for i in range(0, len(scale)):
#     if i == len(scale)-1:
#         freqs += [np.mean(allfreqs[int(scale[i]):])]
#     else:
#         freqs += [np.mean(allfreqs[int(scale[i]):int(scale[i+1])])]

In [ ]:
# freqs

In [ ]:
# ims = 20.*np.log10(np.abs(newspec)/10e-6)  # amplitude to decibel

In [ ]:
# ims

In [ ]:
# np.shape(ims)

In [ ]:
# ims = np.transpose(ims)
# ims = np.flipud(ims)  # weird - not sure why it needs flipping

In [ ]:
# image = Image.fromarray(ims)
# image = image.convert('L')  # convert to grayscale
# image.save("300_AUDIO_8000.png")

In [ ]:
# ims